<a href="https://colab.research.google.com/github/krishnamalani1164/xai-intrusion-detection-shap/blob/main/xai_intrusion_detection_shap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Overview
This project demonstrates the application of Explainable AI techniques to security applications,
specifically focusing on Network Intrusion Detection Systems (IDS). The goal is to create
machine learning models that can effectively detect network intrusions while providing
transparent explanations for their decisions.

## Key Components
1. Data Processing: Using the NSL-KDD dataset, a benchmark dataset for intrusion detection
2. Model Training: Implementation of XGBoost classifier for attack detection
3. Explainable AI: Application of SHAP and LIME techniques to explain model predictions
4. Security Analysis: Evaluation of model performance and robustness against adversarial inputs

#Installation of Dependicies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import warnings
import shap
warnings.filterwarnings("ignore")

#Assigning column names

In [2]:
from google.colab import files
uploaded = files.upload()

Saving NSL-KDD  Dataset.zip to NSL-KDD  Dataset.zip


# Step 2: Unzip the file

In [3]:
import zipfile
import os

zip_path = next(iter(uploaded))  # Get the uploaded zip filename
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()  # Extract into current working directory

In [4]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [5]:
# Assigning attribute name to dataset
dataset_train = pd.read_csv("KDDTrain+.txt", header=None, names=col_names)
dataset_test = pd.read_csv("KDDTest+.txt", header=None, names=col_names)

# Step 5: Display a few rows to confirm
print("Train Dataset:")
print(dataset_train.head())

print("\nTest Dataset:")
print(dataset_test.head())

Train Dataset:
  duration protocol_type service  flag  src_bytes  dst_bytes  land  \
0      tcp      ftp_data      SF   491          0          0     0   
0      udp         other      SF   146          0          0     0   
0      tcp       private      S0     0          0          0     0   
0      tcp          http      SF   232       8153          0     0   
0      tcp          http      SF   199        420          0     0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0               0       0    0  ...                0.17   
0               0       0    0  ...                0.00   
0               0       0    0  ...                0.10   
0               0       0    0  ...                1.00   
0               0       0    0  ...                1.00   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                    0.03                    0.17   
0                    0.60                    0.88   
0                    0.05                    0.00 

In [6]:
#Column that are categorical and not binary yet:
#Explore categorical Features
print('Training set:')
for col_name in dataset_train.columns:
  if dataset_train[col_name].dtypes == 'object':
    unique_cat = len(dataset_train[col_name].unique())
    print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))
#see how distributed the feature service is, it is evenly distributed and therefore we need to make dummies for all.
print()
print('Distribution of categories in service:')
print(dataset_train['service'].value_counts().sort_values(ascending=False).head())


Training set:
Feature 'duration' has 3 categories
Feature 'protocol_type' has 70 categories
Feature 'service' has 11 categories
Feature 'dst_host_srv_rerror_rate' has 23 categories

Distribution of categories in service:
service
SF      74945
S0      34851
REJ     11233
RSTR     2421
RSTO     1562
Name: count, dtype: int64
